# YOLOv11 Fine-tune cho PvZ Bot

Train model nhận diện: sun, zombie, pea_shooter, seed_pack

**Yêu cầu:** GPU Runtime (Runtime > Change runtime type > T4 GPU)

## 1. Cài đặt

In [ ]:
!pip install roboflow ultralytics -q

## 2. Download Dataset từ Roboflow

Thay `YOUR_API_KEY` và project info của bạn

In [ ]:
from roboflow import Roboflow

# === THAY ĐỔI THÔNG TIN NÀY ===
API_KEY = "YOUR_API_KEY"  # Lấy từ roboflow.com/settings
WORKSPACE = "your-workspace"
PROJECT = "pvz-bot-vision"
VERSION = 1
# ==============================

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
dataset = project.version(VERSION).download("yolov11")

print(f"\n✓ Dataset downloaded to: {dataset.location}")

## 3. Train YOLOv11

In [ ]:
from ultralytics import YOLO

# Load YOLOv11 nano (lightweight, fast)
model = YOLO('yolo11n.pt')

# Train
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='pvz_yolo11'
)

print("\n✓ Training complete!")

## 4. Test Model

In [ ]:
import glob
from IPython.display import Image, display

# Load best model
best_model = YOLO('/content/runs/detect/pvz_yolo11/weights/best.pt')

# Test on validation images
val_images = glob.glob(f"{dataset.location}/valid/images/*.jpg")[:3]

for img_path in val_images:
    results = best_model.predict(img_path, save=True, conf=0.5)
    
# Show results
for img in glob.glob('/content/runs/detect/predict/*.jpg')[:3]:
    display(Image(filename=img, width=600))

## 5. Export sang OpenVINO

Tối ưu cho Intel CPU

In [ ]:
# Export to OpenVINO format
best_model.export(format='openvino')

print("\n✓ Exported to OpenVINO!")

## 6. Download Model

In [ ]:
# Zip model for download
!zip -r pvz_yolo_openvino.zip /content/runs/detect/pvz_yolo11/weights/best_openvino_model/

# Also zip the .pt file
!cp /content/runs/detect/pvz_yolo11/weights/best.pt /content/pvz_yolo_best.pt

print("\n✓ Download files:")
print("  - pvz_yolo_openvino.zip (for inference)")
print("  - pvz_yolo_best.pt (original model)")

In [ ]:
from google.colab import files

# Download OpenVINO model
files.download('pvz_yolo_openvino.zip')